# Perceptron - Reminder

* A basic neural network made up of **one or more McCullocha-Pitts nerons**,
* Simply multiplying inputs by weights and summing them up. After sumation result is squeezed to 0-1 range by activation function,
* Linear classifier - usefull only for linear spearable objects,
* **Training**
    * Training - Delta rule algorithm based on Gradient descent,
    * Training can be divided into two steps:
        * Forwad step,
        * Backward step,
    * Loss function:
        * Mean Squared Error - Difference between expected and actual output. Easy derivation, appropriate for regression task
        * Cross entropy - **Kullback–Leibler** divergence between $P(Y | X)$ of data and $Q(Y | X)$ model. Better gradient flow [https://jamesmccaffrey.wordpress.com/2013/11/05/why-you-should-use-cross-entropy-error-instead-of-classification-error-or-mean-squared-error-for-neural-network-classifier-training/]
* Learning types:
    * *Epoch - When we will iterate through a full set of data,*
    * Update weigths every sample,
    * Update weights every batch,
    * Update weigths every epoch (full iteration over dataset).

# Neural network basics

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABCMeta, abstractmethod

First neccessary libraries are imported. In this exercise only numpy and matplolib will be used. <br>
In casual **Deep Learning task** other libraries are used for neural network implementations. <br>
The most popular ones are: <br>
* Tensorflow,
* PyTorch,
* Caffe,
* Keras (*high level*),

The biggest adventages of using these libraries is fact that all of them using computational graph, which can be used for computing on GPU.

![title](imgs/N2.jpg)

## 1.1 Neural network - introduction

An ANN is based on a collection of connected units or nodes called **artificial neurons** (a simplified version of biological neurons in an animal brain). Each connection (a simplified version of a synapse) between artificial neurons can transmit a signal from one to another. The artificial neuron that receives the signal can process it and then signal artificial neurons connected to it.

In common ANN implementations, the signal at a connection between artificial neurons is a real number, and the output of each artificial neuron is calculated by a non-linear function of the sum of its inputs. Artificial neurons and connections typically have a weight that adjusts as learning proceeds. The weight increases or decreases the strength of the signal at a connection. Artificial neurons may have a threshold such that only if the aggregate signal crosses that threshold is the signal sent. Typically, artificial neurons are organized in layers. Different layers may perform different kinds of transformations on their inputs. Signals travel from the first (input), to the last (output) layer, possibly after traversing the layers multiple times.



![title](imgs/neuron.png)

![title](imgs/neuron_model.jpeg)

## 1.2 Neural Network - dictionary

* **Neuron** - The smallest unit in neural network. Represent operation of sum all inputs and use activation function on it.
* **Layer** - Group of neurons. Also represent operation of summing ang activating but can be realised by easi matrix operation.


![title](imgs/neural_eq.png)

* **Activation function** - activation function is non-linearity applied to sum of inputs in neural network.
* **Loss function** - function according to which we will minimise network error.
* **Optimizer** - Algorithm used for optimalization. The best known algorithm for optimalization used is neural networks is Gradient Descent algorithm, but after many years of NN development people discovered many improvements for that like **Nesterov-Momentum**, **Adaptive Gradient** or **Adaptive Momentum**. 
* **Backpropagation** - Best known method for learning Neural Networks.

## 1.3 Neural Network - learning, backpropagation

Same as for perceptron algorithm, which is the simplest neural network, training neural network with hidden <br>layers (*feed forward neural network*) can be also divided into two steps:
* Forward step,
* Backward step.

Only difference is the fact that here we have hidden layers, which weights also should be updated. <br>

In [2]:
class NeuralNetwork(object):
    __metaclass__ = ABCMeta
    
    def __init__(self):
        self.layers = []
        self.loss_function = None
        
    @abstractmethod
    def forward(self):
        pass
    
    @abstractmethod
    def backward(self):
        pass

Define activation function and its derivative. In this exercise sigmoid activation function will be used.

![title](imgs/sigmoid.png)

In Neural Networks with **hidden layers** activation function plays slightly different role than in perceptron. Besides squeezing output to [0, 1] range, activation function introduces non-linearity to the algorithm.
Whithout activation many layers could be reduced to single martix multiplication, which would be the same as linear regression classifier.

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

### 1.3.1 Backpropagation

#### 1.3.1.1 Backpropagation for last layer

The backpropagation algorithm was originally introduced in the 1970s, but its importance wasn't fully appreciated until a famous 1986 paper by David Rumelhart, Geoffrey Hinton, and Ronald Williams. That paper describes several neural networks where backpropagation works far faster than earlier approaches to learning, making it possible to use neural nets to solve problems which had previously been insoluble 


**Backpropagation algorithm is based on propagating error through the network in the same manner like propagating input, only difference is that we are doing it from the end to beginning.**

**On the beginning lets think how we can calculate error** <br>
The most obvious solution is subtracting expected output from actual output and take a power of it - **MSE**

![title](imgs/mse.png)

Now we can calculate global error for model as a whole, but to correct weights we need to know error, which comes from specific branch/weight, for that purpose we can use chain rule. we can calculate global error for model as a whole, but to correct weights we need to know error, which comes from specific branch/weight, for that purpose we can use chain rule.

![title](imgs/e_total.png)

![title](imgs/backprop_model.png)

**1. First derivative** <br>
In first step we are calculating derivative from cost function.

![title](imgs/der1.png)

**2. Second derivative** <br>
Next we are calculating derivative from activation function.

![title](imgs/der2.png)

**3. Third derivative** <br>
In third step we are calculating derivative from previous layer output. 

![title](imgs/der3.png)

**Concat all together**

![title](imgs/concat.png)

In [4]:
class MSE(object):
    
    def __init__(self):
        self.last_layer = None
    
    def __call__(self, arg):
        self.last_layer = arg
        return self
    
    @staticmethod
    def compute(actual_out, expected_out):
        return 0.5 * np.mean(np.sum(np.power(actual_out - expected_out, 2), axis=1))
    
    def compute_backward(self, expected_out):
        actual_out = self.last_layer.compute_forward()
        return actual_out - expected_out

#### 1.3.1.2 Backpropagation for hidden layers

After calculating error for last layer we should calculate error for **hidden layers**. Gradient flow in hidden layers is little different than for perceptron but calculating it is not hard and it is very similiar to **forward step** in neural network.

![title](imgs/nn-calculation.png)

For above example calculating gradient for weight $w_1$ will have form:

![title](imgs/layerw1.png)

We’re going to use a similar process as we did for the output layer, but slightly different to account for the fact that the output of each hidden layer neuron contributes to the output (and therefore error) of multiple output neurons. 

![title](imgs/oh1.png)

![title](imgs/o1.png)

**Concatenating all together**

![title](imgs/backprop12.png)

After fast examination we can see that error, which comes from $w_1$ is nothing different than **summed error multiplied by weights on each branch** from previous layer, after than **multiplied by activation derivative and layer input**. 

This formula can be implemented in each layer allowing to implement layers in form of objects.

In [5]:
class Dense(object):

    def __init__(self, out_shape):
        self.weights = None
        self.biases = None
        self.last_layer = None
        self.layer_shape = None
        self.layer_output = None
        self.layer_input = None
        self.adjustments = None
        self.out_shape = out_shape

    def __call__(self, arg):
        self.build_layer(arg)
        return self

    def build_layer(self, last_layer):
        self.last_layer = last_layer
        self.layer_shape = [last_layer.out_shape, self.out_shape]
        self.weights = np.random.normal(size=self.layer_shape, scale=0.1)
        self.biases = np.random.normal(size=(1, self.out_shape), scale=0.1)

        # Adjustments buffer
        self.adjustments = np.zeros(self.layer_shape)

    def compute_forward(self):
        self.layer_input = self.last_layer.compute_forward()
        net_out = np.dot(self.layer_input.transpose(), self.weights) + self.biases
        self.layer_output = sigmoid(net_out)
        return self.layer_output.transpose()

    def compute_backward(self, prev_deltas):
        prev_deltas_sum = np.dot(self.weights, prev_deltas)
        act_deriative = sigmoid_derivative(self.last_layer.layer_output).T
        new_deltas = np.multiply(prev_deltas_sum, act_deriative)
        return new_deltas

### 1.3.2 Implementing model

After defining layer object, which is able to calculate forward and backward step through the layer it is time to implement Neural Network model. Model will propagate information and gradient through all network and correct the weights.

![title](imgs/join.jpeg)

### 1.3.2.1 Define model object

First we should define model class, which will be used for creating **feed-forward** network.

In model we have 5 important methods:
* compile - collect all layers into list
* forward - perform information flow through network,
* backward - perform gradient flow through network,
* gradient descent - update weigths using ealier computed adjustments,
* train - method used for training network.

In [6]:
class FeedForwardNN(NeuralNetwork):

    def __call__(self, arg):
        self.loss_function = arg
        return self

    def compile(self):
        l_layer = self.loss_function
        while l_layer is not None:
            self.layers.append(l_layer.last_layer)
            l_layer = l_layer.last_layer
        self.network_in = self.layers[-2]
        self.layers = self.layers[:-2][::-1]

    def forward(self):
        prediction = self.layers[-1].compute_forward()
        return prediction

    def backward(self, expected_out):
        deltas = {}
        no_layers = len(self.layers)
        deltas[no_layers] = self.loss_function.compute_backward(expected_out)
        # Calculate deltas
        for l_idx in reversed(range(1, no_layers)):
            layer = self.layers[l_idx]
            deltas[l_idx] = layer.compute_backward(deltas[l_idx + 1])

        # Calculate adjustments
        for l_idx, layer in enumerate(self.layers):
            layer.adjustments += np.dot(deltas[l_idx + 1], layer.layer_input.T).T

    def gradient_descent(self, batch_size, learning_rate):
        # Calculate partial derivative and take a step in that direction
        for layer in self.layers:
            partial_d = (1 / float(batch_size)) * layer.adjustments
            layer.weights += learning_rate * -partial_d
            layer.biases += learning_rate * 1e-3 * -partial_d[-1, :]

    def train(self, inputs_x, targets, num_epochs, learning_rate=0.01, stop_accuracy=1e-5):
        error = []
        for iteration in range(num_epochs):
            for i in range(len(inputs_x)):
                x = inputs_x[i]
                y = targets[i]
                # Pass the training set through our neural network
                self.network_in(x)
                output = self.forward()
                self.backward(expected_out=y)

                # Calculate the error
                loss = MSE.compute(output, y)
                error.append(loss)
                print("Loss function error {}".format(np.mean(error[-(i + 1):])))
            self.gradient_descent(i, learning_rate)
            if np.mean(error[-(i + 1):]) < stop_accuracy and iteration > 0:
                break

In order to standardize the methods, we define the object of entry to the model. Model entry will keep information about input shape and implement methods used in propagation.

![title](imgs/input.png)

In [7]:
class Input(object):

    def __init__(self, shape=2):
        self.input_data = None
        self.out_shape = shape
        self.last_layer = None

    def __call__(self, args):
        self.input_data = args
        return self

    def compute_forward(self):
        return self.input_data

## 1.4 Run model and examine results

On the end we can finaly run our model and examine its results. First problem is known us **XOR** problem, it looks easy but it was very hard task for ealier classifiers. That tas was so hard due to the fact that object, which we should classify are not linearly separable.

In [8]:
training_data = np.asarray([[0, 0], [0, 1], [1, 0], [1, 1]]).reshape(4, 2, 1)
training_labels = np.asarray([[0], [1], [1], [0]])

in_l = Input()
dd_1 = Dense(out_shape=50)(in_l)
dd_2 = Dense(out_shape=125)(dd_1)
dd_3 = Dense(out_shape=1)(dd_2)
mse = MSE()(dd_3)
ff = FeedForwardNN()(mse)
ff.compile()
ff.train(inputs_x=training_data, targets=training_labels, num_epochs=150000, learning_rate=0.01)

Loss function error 0.150354818108
Loss function error 0.126377709161
Loss function error 0.118407664699
Loss function error 0.126123280582
Loss function error 0.147733254651
Loss function error 0.126158174431
Loss function error 0.118989408402
Loss function error 0.125907452418
Loss function error 0.142807571854
Loss function error 0.125803876519
Loss function error 0.120160062406
Loss function error 0.125560314582
Loss function error 0.13616731393
Loss function error 0.12545253848
Loss function error 0.121906643339
Loss function error 0.125219031725
Loss function error 0.128586482729
Loss function error 0.125241393308
Loss function error 0.124153974502
Loss function error 0.12501992783
Loss function error 0.120906716204
Loss function error 0.125251869357
Loss function error 0.126729883057
Loss function error 0.125043207996
Loss function error 0.113918070868
Loss function error 0.125475951653
Loss function error 0.129360112093
Loss function error 0.125279467017
Loss function error 0.1

Loss function error 0.132628048605
Loss function error 0.125805768098
Loss function error 0.110926951527
Loss function error 0.125632713524
Loss function error 0.130567383073
Loss function error 0.125447264143
Loss function error 0.117282197871
Loss function error 0.125334936625
Loss function error 0.128050059634
Loss function error 0.125138478973
Loss function error 0.124662774037
Loss function error 0.125210889382
Loss function error 0.125422363693
Loss function error 0.125002136742
Loss function error 0.132350779345
Loss function error 0.125313665444
Loss function error 0.1229946156
Loss function error 0.125092693906
Loss function error 0.139528594414
Loss function error 0.125605544472
Loss function error 0.120989284868
Loss function error 0.125373730305
Loss function error 0.145384620092
Loss function error 0.12597346135
Loss function error 0.119526337939
Loss function error 0.12573323449
Loss function error 0.149232442762
Loss function error 0.126274977806
Loss function error 0.11

Loss function error 0.125617931524
Loss function error 0.148130042465
Loss function error 0.12617098732
Loss function error 0.118873348638
Loss function error 0.125942249513
Loss function error 0.150296393265
Loss function error 0.126353504038
Loss function error 0.118393975856
Loss function error 0.126122129439
Loss function error 0.149911411144
Loss function error 0.126319698211
Loss function error 0.118477322063
Loss function error 0.126089177036
Loss function error 0.147022536131
Loss function error 0.126082287115
Loss function error 0.119124528192
Loss function error 0.125856068655
Loss function error 0.141971176468
Loss function error 0.125730557584
Loss function error 0.120340789788
Loss function error 0.125511812764
Loss function error 0.135344641888
Loss function error 0.125397522793
Loss function error 0.122107579633
Loss function error 0.125188860237
Loss function error 0.12789070855
Loss function error 0.125209454273
Loss function error 0.124343828664
Loss function error 0.

Loss function error 0.129076888946
Loss function error 0.125248426074
Loss function error 0.12115917261
Loss function error 0.125195045948
Loss function error 0.126571827705
Loss function error 0.125036354946
Loss function error 0.128461034249
Loss function error 0.125188597593
Loss function error 0.124126228384
Loss function error 0.125019109049
Loss function error 0.135661285968
Loss function error 0.12538244833
Loss function error 0.121981747636
Loss function error 0.125202826365
Loss function error 0.142014780761
Loss function error 0.125705990552
Loss function error 0.120292892319
Loss function error 0.125517904567
Loss function error 0.146833460409
Loss function error 0.126040492595
Loss function error 0.119130904903
Loss function error 0.125846370964
Loss function error 0.149582815529
Loss function error 0.126264017503
Loss function error 0.118511498208
Loss function error 0.12606676897
Loss function error 0.149955764444
Loss function error 0.126295770013
Loss function error 0.1

Loss function error 0.132675291285
Loss function error 0.125824028113
Loss function error 0.103547606794
Loss function error 0.126124399272
Loss function error 0.133694447657
Loss function error 0.126033580353
Loss function error 0.10318085445
Loss function error 0.126160086734
Loss function error 0.133864603281
Loss function error 0.126070430629
Loss function error 0.104696846463
Loss function error 0.126013721783
Loss function error 0.133163186912
Loss function error 0.125922157717
Loss function error 0.108001388471
Loss function error 0.125736589844
Loss function error 0.131690065808
Loss function error 0.125640277717
Loss function error 0.112878846607
Loss function error 0.125426325192
Loss function error 0.129647506841
Loss function error 0.125322918509
Loss function error 0.118978944997
Loss function error 0.125193397903
Loss function error 0.12729987482
Loss function error 0.125081280183
Loss function error 0.125814524764
Loss function error 0.125122002099
Loss function error 0.

...

Loss function error 0.08359944342
Loss function error 0.106275397808
Loss function error 0.10905576533
Loss function error 0.0135488569419
Loss function error 0.0835484369197
Loss function error 0.106380408096
Loss function error 0.109078500127
Loss function error 0.0130047100608
Loss function error 0.0835058843195
Loss function error 0.106488197657
Loss function error 0.109103875096
Loss function error 0.0124877936191
Loss function error 0.0834697405106
Loss function error 0.106595692887
Loss function error 0.109130283388
Loss function error 0.0119994173389
Loss function error 0.0834376101542
Loss function error 0.106699446158
Loss function error 0.109155906862
Loss function error 0.0115406482862
Loss function error 0.0834067902367
Loss function error 0.106795694224
Loss function error 0.109178747846
Loss function error 0.011112304813
Loss function error 0.083374331697
Loss function error 0.106880438657
Loss function error 0.109196672971
Loss function error 0.0107149576496
Loss functi

Loss function error 0.0734576902003
Loss function error 0.0943099977826
Loss function error 0.00677346533711
Loss function error 0.0556780581817
Loss function error 0.071684642488
Loss function error 0.0938436352216
Loss function error 0.00642495402122
Loss function error 0.0542102033825
Loss function error 0.0698401770283
Loss function error 0.0933859944651
Loss function error 0.00607553920285
Loss function error 0.0526976695185
Loss function error 0.0679367637688
Loss function error 0.092941867022
Loss function error 0.00572714864223
Loss function error 0.0511521578046
Loss function error 0.0659893127252
Loss function error 0.0925157811678
Loss function error 0.00538170161022
Loss function error 0.049586840407
Loss function error 0.0640146552642
Loss function error 0.0921117823828
Loss function error 0.00504109181593
Loss function error 0.0480158424671
Loss function error 0.0620308615548
Loss function error 0.0917332622075
Loss function error 0.00470716703146
Loss function error 0.04

Loss function error 0.0367652064579
Loss function error 0.0624375060185
Loss function error 0.000170927109472
Loss function error 0.0267387233482
Loss function error 0.0356093864929
Loss function error 0.0611446488064
Loss function error 0.000173425451706
Loss function error 0.0261615136531
Loss function error 0.0348985197368
Loss function error 0.0597379266347
Loss function error 0.00017566353305
Loss function error 0.0259125514168
Loss function error 0.0346246650503
Loss function error 0.0582140933337
Loss function error 0.000177710392601
Loss function error 0.0259527581305
Loss function error 0.0347377135985
Loss function error 0.0566040658217
Loss function error 0.000179697957361
Loss function error 0.0262067966411
Loss function error 0.0351384065124
Loss function error 0.05496480023
Loss function error 0.000181804593467
Loss function error 0.0265590190588
Loss function error 0.03567246024
Loss function error 0.0533568089314
Loss function error 0.000184230989063
Loss function error

Loss function error 0.00315538182044
Loss function error 0.000240541807112
Loss function error 0.000482314623556
Loss function error 0.000535744660219
Loss function error 0.0030468645841
Loss function error 0.000229904984903
Loss function error 0.000434146373939
Loss function error 0.000477219017329
Loss function error 0.00293864469653
Loss function error 0.000219140932436
Loss function error 0.000392872991404
Loss function error 0.000427649654262
Loss function error 0.00282811876598
Loss function error 0.000208270989501
Loss function error 0.000357594288806
Loss function error 0.000385825448263
Loss function error 0.0027134641234
Loss function error 0.000197330637556
Loss function error 0.000327437462226
Loss function error 0.000350582399062
Loss function error 0.00259375492832
Loss function error 0.000186367602328
Loss function error 0.000301608722515
Loss function error 0.000320866258139
Loss function error 0.0024689612017
Loss function error 0.000175439107413
Loss function error 0.

Loss function error 2.45429320872e-06
Loss function error 2.19083764496e-05
Loss function error 9.74146454262e-08
Loss function error 1.99540200031e-06
Loss function error 2.30150512952e-06
Loss function error 1.94865336559e-05
Loss function error 8.21752029026e-08
Loss function error 1.86929503572e-06
Loss function error 2.16319862186e-06
Loss function error 1.72696656821e-05
Loss function error 6.92726496762e-08
Loss function error 1.75516851351e-06
Loss function error 2.03797457615e-06
Loss function error 1.52477989532e-05
Loss function error 5.83626053521e-08
Loss function error 1.65184624891e-06
Loss function error 1.92460573056e-06
Loss function error 1.34111135026e-05
Loss function error 4.91483433252e-08
Loss function error 1.55829628737e-06
Loss function error 1.82201631299e-06
Loss function error 1.17498954062e-05
Loss function error 4.13748005066e-08
Loss function error 1.47361367633e-06
Loss function error 1.72926446976e-06
Loss function error 1.02544946275e-05
Loss functio